<img src="logo.png" width="420px">

# `TODOIST AUTOMATION`

### Settings

#### Libraries

In [2]:
from todoist_api_python.api import TodoistAPI
import requests
import json
import uuid
from datetime import datetime, timedelta, timezone

#### Dependences

In [3]:
import credentials

#### API

In [4]:
api_token = credentials.api_token
api = TodoistAPI(api_token)

### Data

#### Extraction

In [5]:
headers = {
    "Content-Type": "application/json",
    "X-Request-Id": "un_valor_único_por_cada_solicitud",
    "Authorization": f"Bearer {api_token}"
}

In [6]:
def get_projects():
    data = {
        "sync_token": "*",
        "resource_types": json.dumps(['projects'])
    }
    response = requests.post("https://api.todoist.com/sync/v9/sync", headers=headers, data=json.dumps(data))
    all_projects = response.json().get('projects', [])
    return all_projects

def get_projects_dicts(all_projects):
    projects_dict_id={}
    projects_dict_name={}
    for project in all_projects:
        projects_dict_id[project['id']]=project['name']
        projects_dict_name[project['name']]=project['id']
    return projects_dict_id, projects_dict_name

In [7]:
def get_tasks():
    data = {
        "sync_token": "*",
        "resource_types": json.dumps(['items'])
    }
    response = requests.post("https://api.todoist.com/sync/v9/sync", headers=headers, data=json.dumps(data))
    all_tasks = response.json()["items"]
    return all_tasks

def get_tasks_dicts(all_tasks, projects_dict_id, projects_dict_name):
    task_dict_id={}
    task_dict_name={}
    for task in all_tasks:
        task_dict_name[task['content']]=[task['id'],projects_dict_id[task['project_id']]]
        task_dict_id[task['id']]=[task['content'],projects_dict_id[task['project_id']]]
    return task_dict_id, task_dict_name

In [8]:
def get_sections():
    data = {
        "sync_token": "*",
        "resource_types": json.dumps(['sections'])
    }
    response = requests.post("https://api.todoist.com/sync/v9/sync", headers=headers, data=json.dumps(data))
    all_sections = response.json()["sections"]
    return all_sections

def get_sections_dicts(all_sections):
    sections_dict_id={}
    sections_dict_name={}
    for section in all_sections:
        sections_dict_id[section['id']]=section['name']
        sections_dict_name[section['name']]=section['id']
    return sections_dict_id, sections_dict_name

#### Projects

In [9]:
all_projects = get_projects()
projects_dict_id, projects_dict_name = get_projects_dicts(all_projects)
projects_dict_id

{'2258455386': 'Inbox',
 '2259406345': 'Calendario',
 '2259150181': 'Fútbol',
 '2269361803': 'Fútbol archivados',
 '2259111397': 'Fórmula 1',
 '2298494169': 'Tareas',
 '2259719762': 'Diarias',
 '2259719779': 'Semanales',
 '2259719838': 'Mensuales',
 '2307572337': 'Anuales',
 '2258518194': 'Recordatorios',
 '2259726698': 'Cumpleaños',
 '2323776352': 'PPP',
 '2316607649': 'Publicis',
 '2328028918': 'Sales Hacking',
 '2303925714': 'Cursos',
 '2305010575': 'Allianz',
 '2320233020': 'GitHub',
 '2263759336': 'Incubadora',
 '2258473389': 'Tickler',
 '2263729931': 'Archivados',
 '2323984038': 'b',
 '2273727343': 'Enterrados'}

#### Tasks

In [10]:
all_tasks = get_tasks()
task_dict_id, task_dict_name = get_tasks_dicts(all_tasks, projects_dict_id, projects_dict_name)

In [11]:
task_dict_id

{'4582634104': ['Examen Claudia', 'Enterrados'],
 '4582658679': ['Hacer cronos', 'Archivados'],
 '4584499496': ['* PREMIER LEAGUE 2020/2021', 'Fútbol archivados'],
 '4584503164': ['* UEFA EUROPA LEAGUE 2020/2021', 'Fútbol archivados'],
 '4584503456': ['* EMIRATES FA CUP 2020/2021', 'Fútbol archivados'],
 '4584522108': ['Spotify', 'Mensuales'],
 '4584522265': ['B', 'Enterrados'],
 '4586378739': ['Felicitar Loren', 'Cumpleaños'],
 '4586379074': ['Felicitar Quique', 'Cumpleaños'],
 '4586379605': ['Felicitar Jimmy', 'Cumpleaños'],
 '4587186415': ['Felicitar Mollita', 'Cumpleaños'],
 '4587186606': ['Felicitar Bego', 'Cumpleaños'],
 '4587186943': ['Felicitar Martu', 'Cumpleaños'],
 '4587187228': ['Felicitar Martasa', 'Cumpleaños'],
 '4587187631': ['Felicitar Paulah', 'Cumpleaños'],
 '4587188029': ['Felicitar Mamá', 'Cumpleaños'],
 '4587188363': ['Felicitar Papá', 'Cumpleaños'],
 '4587189250': ['Felicitar Puru', 'Cumpleaños'],
 '4587189625': ['Felicitar Martín', 'Cumpleaños'],
 '4587190027': 

#### Sections

In [12]:
all_sections = get_sections()
sections_dict_id, sections_dict_name = get_sections_dicts(all_sections)

In [13]:
sections_dict_id

{'51988025': 'MANCHESTER UNITED',
 '54450645': 'Otros',
 '132796518': 'UEFA CHAMPIONS LEAGUE 2023/2024',
 '138032252': 'UEFA EURO 2024',
 '54450661': 'UEFA CHAMPIONS LEAGUE 2021_2022',
 '51987985': 'UEFA EURO 2020',
 '83399834': 'MANCHESTER UNITED',
 '85265725': 'FIFA WORLD CUP 2022',
 '99652236': 'UEFA CHAMPIONS LEAGUE 2022_2023',
 '135108411': 'Eventos',
 '135108636': 'Recordatorios',
 '137136088': 'SMARTs',
 '137240160': 'Tareas',
 '137327232': 'SMARTpro',
 '137327248': 'SMARTex',
 '138608878': 'Keyword Generator',
 '139453198': 'Auction Insights',
 '140074689': 'Keyword Tracking',
 '140961721': 'Facturación',
 '134816386': 'JFK Delay Prediction',
 '134816473': 'CIFAR-10',
 '134816489': 'Animal Classification App',
 '134816578': 'Times Up',
 '134816588': 'Microsoft Malware Detection',
 '136725407': 'Educational Analysis PISA',
 '138439288': 'Blockchain Creation',
 '142986758': 'Todolist',
 '149422833': 'Todoist',
 '149457958': 'Expenses',
 '140324507': 'Investigar',
 '140324373': 'S

#### Test

In [14]:
index=150
try:
    print(all_tasks[index]['content'], "-", all_tasks[index]['due']['date'], "-", projects_dict_id[all_tasks[index]['project_id']])
except TypeError:
    print(all_tasks[index]['content'], "-", "No date" , "-",projects_dict_id[all_tasks[index]['project_id']])

Plantilla Top Eleven - No date - Enterrados


#### Labels

In [15]:
label_names=[]
for task in all_tasks:
    gross_labels=task['labels']
    for label in gross_labels:
        if label not in label_names:
            label_names.append(label)
label_names

['Med',
 'PC',
 'Móvil',
 'Short',
 'Casa',
 'Long',
 'Fuera',
 '❌_streak',
 '✅_streak',
 'Recurrente',
 'Vacaciones']

In [16]:
try:
    is_success = api.update_task(task_id="7802279241", content="Buy Coffee")
    print(is_success)
except Exception as error:
    print(error)

404 Client Error: Not Found for url: https://api.todoist.com/rest/v2/tasks/7802279241


### `Orders`

#### Creating tasks

In [17]:
def priority_inversal(n):
    if n==1:
        return 4
    if n==3:
        return 2
    if n==2:
        return 3
    if n==4:
        return 1

In [34]:
def create_task(
    assignee_id = None,
    comment_count = 0,
    is_completed = False,
    content = "No se añadio título",
    description = "",
    date = None,
    is_recurring = False,
    datetime = None,
    datestring = None,
    duration = None,
    labels = [],
    order = 1,
    priority = 4,
    project_id = projects_dict_name['Inbox'],
    section_id = None,
    parent_id = None,
):
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.add_task(
            assignee_id = assignee_id,
            comment_count = comment_count,
            is_completed = is_completed,
            content = content,
            description = description,
            due = {'date':date,
                   'is_recurring':is_recurring,
                   'datetime':datetime,
                   'datestring':datestring},
            duration = duration,
            labels = labels,
            order = order,
            priority = priority_inversal(priority),
            project_id = project_id,
            section_id = section_id,
            parent_id = parent_id,
        )
        print(f'La tarea {task.content} se creó correctamente. Id = {task.id}')
    except Exception as error:
        print(error)

In [28]:
create_task(content='Pruea',
            labels=['Short'],
            priority=1,
            description="Descripción de prueba",
            date="2024-04-01"
)
all_tasks = get_tasks()
task_dict_id, task_dict_name = get_tasks_dicts(all_tasks, projects_dict_id, projects_dict_name)

La tarea Pruargregebaaaa se creó correctamente. Id = 7802620088


#### Editing tasks

In [50]:
def edit_task(
    id,
    assignee_id = None,
    comment_count = 0,
    is_completed = False,
    description = "",
    date = None,
    is_recurring = False,
    datetime = None,
    datestring = None,
    duration = None,
    labels = [],
    order = 1,
    priority = 4,
    project_id = projects_dict_name['Inbox'],
    section_id = None,
    parent_id = None,
):
    for label in labels:
        if label not in label_names:
            return f'La etiqueta {label} no existe'
    try:
        task = api.update_task(
            task_id = id,
            assignee_id = assignee_id,
            comment_count = comment_count,
            is_completed = is_completed,
            content = task_dict_id[id][0],
            description = description,
            due = {'date':date,
                   'is_recurring':is_recurring,
                   'datetime':datetime,
                   'datestring':datestring},
            duration = duration,
            labels = labels,
            order = order,
            priority = priority_inversal(priority),
            project_id = project_id,
            section_id = section_id,
            parent_id = parent_id,
        )
        print(f'La tarea se editó correctamente')
    except Exception as error:
        print(error)

In [51]:
create_task(content='Esta tarea será editada',
            labels=['Short'],
            priority=1,
            description="jaja",
            date="2024-04-01"
)

all_tasks = get_tasks()
task_dict_id, task_dict_name = get_tasks_dicts(all_tasks, projects_dict_id, projects_dict_name)

La tarea Esta tarea será editada se creó correctamente. Id = 7802642829


In [52]:
edit_task(id='7802642829',
          priority=2,
          description='Ya ha sido editada de nuevo')


all_tasks = get_tasks()
task_dict_id, task_dict_name = get_tasks_dicts(all_tasks, projects_dict_id, projects_dict_name)

La tarea se editó correctamente
